# Timer

In [14]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 9, 17, 30

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [15]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '30m'
higher_timeframe = '6h'
opt_limit = 50
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 30m/6h, trade type is buy
Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:46<00:00, 11.34s/it]


# Check local statistics

In [3]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
0,PriceChange,1,100.00,75.00,75.00,87.50,75.00,62.50,62.50,62.50,62.50,62.50,62.50,62.50,50.00,62.50,62.50,62.50,75.00,62.50,75.00,50.00,37.50,25.00,37.50,37.50,5.15,1.42,1.30,1.57,2.51,1.39,2.27,3.75,3.10,2.66,3.43,4.38,1.88,1.18,0.65,0.78,5.17,1.64,0.46,0.18,-0.93,-2.61,-1.95,-2.88,8,61.979167,1.520833,-64.166667,12.166667
1,PriceChange,2,100.00,80.00,72.00,72.00,76.00,68.00,68.00,64.00,60.00,56.00,56.00,64.00,56.00,60.00,60.00,60.00,68.00,60.00,56.00,52.00,48.00,44.00,36.00,40.00,2.76,1.27,0.97,0.99,1.24,1.05,0.84,0.96,0.73,0.26,0.81,0.69,1.49,0.69,0.69,1.01,2.38,0.57,1.05,0.46,-0.54,-1.73,-2.21,-2.33,25,61.500000,0.587500,-212.500000,14.687500
4,PriceChange,5,98.15,75.00,80.56,67.59,83.33,77.78,75.93,74.07,62.96,62.04,65.74,70.37,74.07,67.59,66.67,64.81,72.22,68.52,64.81,64.81,50.93,38.89,35.19,30.56,1.31,0.64,1.22,0.50,1.72,1.31,1.45,1.32,0.73,0.47,1.17,1.64,2.04,1.05,1.19,1.24,1.90,0.91,1.02,1.17,0.12,-0.58,-1.16,-1.57,108,66.357917,0.867083,-393.345000,93.645000
3,PriceChange,4,100.00,76.00,78.67,69.33,81.33,76.00,73.33,70.67,61.33,58.67,64.00,66.67,69.33,62.67,62.67,60.00,69.33,65.33,61.33,60.00,46.67,37.33,32.00,28.00,1.82,0.75,1.22,0.53,1.77,1.30,1.45,1.32,0.73,0.46,1.06,1.64,2.15,1.05,1.10,1.01,1.84,0.84,0.97,0.95,-0.20,-0.72,-1.22,-1.93,75,63.777500,0.828750,-466.687500,62.156250
5,PriceChange,6,98.64,71.43,78.91,66.67,82.31,75.51,78.23,75.51,62.59,61.90,68.03,74.15,75.51,68.71,69.39,68.03,73.47,70.07,68.71,65.99,50.34,37.41,33.33,28.57,1.23,0.54,0.97,0.43,1.48,1.26,1.40,1.32,0.72,0.43,1.12,1.56,1.88,1.06,1.19,1.01,1.80,0.83,1.05,1.14,0.04,-0.57,-1.18,-1.58,147,66.808750,0.797083,-469.113750,117.171250
6,PriceChange,7,98.38,71.89,80.00,67.03,82.70,77.30,78.92,75.14,64.86,63.24,67.03,74.05,75.14,69.73,69.73,67.57,72.97,68.65,67.57,65.95,49.19,37.30,32.43,27.57,1.21,0.53,0.97,0.46,1.48,1.28,1.40,1.39,0.73,0.43,1.15,1.56,2.02,1.12,1.19,1.01,1.76,0.83,1.03,1.07,-0.03,-0.57,-1.22,-1.58,185,66.847500,0.800833,-583.212500,148.154167
2,PriceChange,3,100.00,79.59,75.51,73.47,73.47,69.39,65.31,57.14,48.98,46.94,51.02,55.10,57.14,55.10,55.10,53.06,63.27,57.14,53.06,48.98,40.82,32.65,26.53,26.53,2.11,1.12,1.22,0.80,1.24,1.28,0.84,0.74,-0.08,-0.76,0.20,0.38,1.06,0.35,0.62,0.51,1.24,0.36,0.27,-0.04,-0.54,-1.37,-2.26,-2.79,49,56.887500,0.270833,-642.512500,13.270833
7,PriceChange,8,98.53,70.10,79.90,66.18,83.33,77.94,79.41,75.00,64.22,62.75,67.16,74.51,75.00,68.63,69.12,66.18,72.06,67.65,66.18,65.20,48.04,36.76,31.37,27.45,1.20,0.52,0.97,0.46,1.46,1.27,1.39,1.32,0.72,0.43,1.14,1.56,1.94,1.05,1.15,0.94,1.64,0.80,0.99,1.05,-0.06,-0.58,-1.25,-1.63,204,66.361250,0.770000,-742.305000,157.080000
8,PriceChange,9,98.31,69.62,80.17,64.98,81.86,76.79,78.06,73.00,63.71,61.60,66.24,72.57,73.00,66.67,67.09,64.56,70.04,65.40,64.14,63.29,45.99,34.60,29.54,26.58,1.19,0.53,0.97,0.42,1.33,1.24,1.31,1.27,0.72,0.43,1.07,1.45,1.79,0.99,1.14,0.83,1.50,0.71,0.88,0.94,-0.10,-0.69,-1.35,-1.69,237,64.908750,0.703

# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,48.911181,0.698576,-44.928403,0.749826,33
2,53.224514,1.224826,-96.531319,0.555521,60
4,63.002222,1.507500,-106.731771,3.426736,123
3,57.661771,1.482188,-108.683125,1.100035,90
5,63.067674,1.100035,-111.523819,7.445278,154
6,63.261736,1.080729,-129.090833,7.653299,176
7,59.737639,0.319965,-155.473542,7.016875,216
8,60.148194,0.825590,-160.278958,10.196632,242
9,60.661910,0.429826,-167.830035,11.735139,278


# Save new config data to config file

In [18]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [7]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)